In [1]:
import requests
import os
import glob
import pandas as pd

In [2]:
# Enter the TASK ID for a FBMN job

GNPS_TASK = "ea1c638c18f94a3d856eecc87d521394"
TIMELINE_AXIS = "timepoint_hour"
INDIVIDUAL_COLUMN = "subject_id"

In [3]:
# Getting the Data

def download_file(url, local_filename):
    # NOTE the stream=True parameter below
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                # If you have chunk encoded response uncomment if
                # and set chunk_size parameter to None.
                #if chunk: 
                f.write(chunk)

task_qza_url = "https://gnps.ucsd.edu/ProteoSAFe/DownloadResultFile?task={}&block=main&file=qiime2_output/qiime2_table.qza".format(GNPS_TASK)
TABLE_QZA = "qiime2_table.qza"
download_file(task_qza_url, TABLE_QZA)

METADATA_TSV = "metadata.tsv"
metadata_url =  "https://gnps.ucsd.edu/ProteoSAFe/DownloadResultFile?task={}&block=main&file=qiime2_output/qiime2_metadata.tsv".format(GNPS_TASK)
download_file(metadata_url, METADATA_TSV)

In [78]:
# OPTIONAL: Cleaning up Metadata File to make sure the timeline axis is numeric

NEW_METADATA_TSV = "metadata_filtered.tsv"
metadata_df = pd.read_csv(METADATA_TSV, sep="\t")
metadata_df = metadata_df[metadata_df[TIMELINE_AXIS].apply(lambda x: x.isdecimal())]
metadata_df.to_csv(NEW_METADATA_TSV, sep="\t", index=False)

METADATA_TSV = NEW_METADATA_TSV
# Filtering the samples if no metadata
FILTERED_TABLE_QZA = "qiime2_metadata_filtered_table.qza"
cmd = "qiime feature-table filter-samples --i-table {} --m-metadata-file {} --o-filtered-table {}".format(TABLE_QZA, METADATA_TSV, FILTERED_TABLE_QZA)
print(cmd)
!$cmd
TABLE_QZA = FILTERED_TABLE_QZA

qiime feature-table filter-samples --i-table qiime2_table.qza --m-metadata-file metadata_filtered.tsv --o-filtered-table qiime2_metadata_filtered_table.qza


In [71]:
# OPTIONAL: Filtering through samples, please upload a file called filtered_metadata.tsv

FILTERING_METADATA = "filtered_metadata.tsv"
FILTERED_TABLE_QZA = "qiime2_filtered_table.qza"
cmd = "qiime feature-table filter-samples --i-table {} --m-metadata-file {} --o-filtered-table {}".format(TABLE_QZA, FILTERING_METADATA, FILTERED_TABLE_QZA)
print(cmd)
!$cmd
TABLE_QZA = FILTERED_TABLE_QZA

qiime feature-table filter-samples --i-table qiime2_metadata_filtered_table.qza --m-metadata-file filtered_metadata.tsv --o-filtered-table qiime2_filtered_table.qza


In [72]:
# Calculating Beta diversity btwn samples —> .qza (PCoA, biplots, permanovas etc)

DISTANCE_QZA = "sample_table_braycurtis.qza"
cmd = "qiime diversity beta --i-table {} --p-metric braycurtis --o-distance-matrix {}".format(TABLE_QZA, DISTANCE_QZA)
print(cmd)
!$cmd

qiime diversity beta --i-table qiime2_filtered_table.qza --p-metric braycurtis --o-distance-matrix sample_table_braycurtis.qza


0

In [73]:
# PCOA Calculation 

PCOA_QZA = "sample_pcoa.qza"
cmd = "qiime diversity pcoa --i-distance-matrix {} --o-pcoa {}".format(DISTANCE_QZA, PCOA_QZA)

print(cmd)
!$cmd

qiime diversity pcoa --i-distance-matrix sample_table_braycurtis.qza --o-pcoa sample_pcoa.qza


0

In [74]:
# Create PCoA visualisation

EMPEROR_QZV = "emperor.qzv"
cmd = "qiime emperor plot --i-pcoa {} --m-metadata-file {} --output-dir {}".format(PCOA_QZA, METADATA_TSV, EMPEROR_QZV)

print(cmd)
!$cmd

qiime emperor plot --i-pcoa sample_pcoa.qza --m-metadata-file metadata.tsv --output-dir emperor.qzv


256

In [75]:
# Create PCoA with Timeline axis visualisation

EMPEROR_TIMELINE_QZV = "emperor_timeline.qzv"
cmd = "qiime emperor plot --i-pcoa {} --m-metadata-file {} --p-custom-axes {} --output-dir {}".format(PCOA_QZA, METADATA_TSV, TIMELINE_AXIS, EMPEROR_TIMELINE_QZV)

print(cmd)
!$cmd

qiime emperor plot --i-pcoa sample_pcoa.qza --m-metadata-file metadata.tsv --p-custom-axes timepoint_hour --output-dir emperor_timeline.qzv


256

In [76]:
# Create Volatility longitudinal plots (p-state-column is timeline column name, p-individual-id is subject id or other connected points)

VOLATILE_FILTERED_QZA = "volatile_filtered_table.qza"
VOLATILE_IMPORTANCE_QZA = "volatile_importance.qza"
VOLATILITY_QZV = "volatile.qzv"
VOLATILITY_ACCURACY_QZV = "volatile_accuracy.qzv"
VOLATILITY_ESTIMATOR_QZA = "valatile_estimator.qza"
cmd = "qiime longitudinal feature-volatility --i-table {} \
--m-metadata-file {} \
--p-state-column {} \
--p-individual-id-column {} \
--o-filtered-table {} \
--o-feature-importance {} \
--o-volatility-plot {} \
--o-accuracy-results {} \
--o-sample-estimator {}".format(TABLE_QZA, METADATA_TSV, TIMELINE_AXIS, INDIVIDUAL_COLUMN, VOLATILE_FILTERED_QZA, VOLATILE_IMPORTANCE_QZA, VOLATILITY_QZV, VOLATILITY_ACCURACY_QZV, VOLATILITY_ESTIMATOR_QZA)

print(cmd)
!$cmd

qiime longitudinal feature-volatility --i-table qiime2_filtered_table.qza --m-metadata-file metadata.tsv --p-state-column timepoint_hour --p-individual-id-column subject_id --o-filtered-table volatile_filtered_table.qza --o-feature-importance volatile_importance.qza --o-volatility-plot volatile.qzv --o-accuracy-results volatile_accuracy.qzv --o-sample-estimator valatile_estimator.qza


2